<a href="https://colab.research.google.com/github/RutherfordOtu12/Application-of-GeoAI-A-Comparative-Methodical-urban-mapping-/blob/main/Validation_points_stratified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Point

# Load clipped land use data
fk_landuse = gpd.read_file('FK_landuse_2020_clipped.gpkg')

# Define target points per class (stratified)
target_points = {
    10: 135,      # Residential (40-50%)
    21: 30,       # Mixed-Use (8-12%)
    30: 30,       # Commercial (part of Class 3)
    40: 30,       # Commercial/Industrial (part of Class 3)
    50: 22,       # Public/Institutional (5-10%)
    60: 15,       # Utilities (part of Class 3)
    70: 10,       # Weekend houses (part of Class 5)
    80: 22,       # Transportation (5-10%)
    100: 8,       # Forest (part of Class 5)
    121: 5,       # Grassland (part of Class 5)
    122: 2,       # Arable land (part of Class 5)
    130: 10,      # Parks (part of Class 5)
    150: 3,       # Cemeteries (part of Class 5)
    160: 5,       # Allotment gardens (part of Class 5)
    200: 2,       # Tree nurseries (part of Class 5)
    140: 6,       # Squares (part of Class 3 or 5)
    90: 2,        # Construction (excluded)
    110: 1,       # Water (excluded)
    171: 1,       # Fallow (excluded)
}

# Generate stratified random points
validation_points = []
point_id = 1

for code, target_count in target_points.items():
    # Filter polygons with this code
    class_polygons = fk_landuse[fk_landuse['code'] == code]

    if len(class_polygons) == 0:
        continue

    # Generate random points within these polygons
    for _ in range(target_count):
        # Select random polygon
        polygon = class_polygons.sample(1).geometry.iloc[0]

        # Generate random point within polygon
        minx, miny, maxx, maxy = polygon.bounds
        while True:
            x = np.random.uniform(minx, maxx)
            y = np.random.uniform(miny, maxy)
            point = Point(x, y)
            if polygon.contains(point):
                break

        validation_points.append({
            'point_id': point_id,
            'official_code': code,
            'geometry': point
        })
        point_id += 1

# Create GeoDataFrame
validation_gdf = gpd.GeoDataFrame(validation_points, crs='EPSG:25833')

# Save
validation_gdf.to_file('FK_validation_points_300.shp')
validation_gdf.to_file('FK_validation_points_300.gpkg', driver='GPKG')

print(f"Generated {len(validation_gdf)} validation points")
print(validation_gdf['official_code'].value_counts())